In [162]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import f1_score, accuracy_score
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, cv, Pool
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [163]:
all_data = pd.read_csv('Credit_card.csv')
labels = pd.read_csv('Credit_card_label.csv')
all_data = pd.merge(all_data, labels, left_on='Ind_ID', right_on='Ind_ID')
all_data.drop(columns = ['Ind_ID', 'Type_Occupation', 'Birthday_count', 'Mobile_phone', 'Work_Phone', 'Phone', 'EMAIL_ID'], inplace=True)
all_data.dropna(inplace = True)

In [164]:
all_data.head()

,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Employed_days,Family_Members,label
0,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,365243,2,1
1,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-586,2,1
2,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-586,2,1
4,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-586,2,1
6,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-586,2,1


### [Dataset Source](https://www.kaggle.com/datasets/rohitudageri/credit-card-details?select=Credit_card_label.csv)


### Attribute Information:
<ul>
<li><b>Gender</b>: Gender information</li>
<li><b>Car_owner</b>: Having car or not</li>
<li><b>Propert_owner</b>: Having property or not</li>
<li><b>Children</b> Count of children</li>
<li><b>Annual_income</b>: Annual income</li>
<li><b>Type_Income</b>: Income type</li>
<li><b>Education</b>: Education level</li>
<li><b>Marital_status</b>: Marital_status</li>
<li><b>Housing_type</b>: Living style</li>
<li><b>Birthday_count</b>: Use backward count from current day (0), -1 means yesterday</li>
<li><b>Employed_days</b>: Start date of employment. Use backward count from current day (0). Positive value means, individual is currently unemployed</li>
<li><b>Family_Members</b>: Family size</li>
<li><b>Label</b>: 0 is application approved and 1 is application rejected</li>

In [165]:
s = (all_data.dtypes == 'object')
object_cols = list(s[s].index)
ordinal_encoder = OrdinalEncoder(dtype = int)
tmp = all_data.copy()
tmp[object_cols] = ordinal_encoder.fit_transform(tmp[object_cols])

X, y = tmp.drop(columns = 'label'), tmp['label']

In [166]:
skf = StratifiedKFold(shuffle = True, random_state=42)

## Training the models

In [167]:
results = pd.DataFrame(columns = ['f1', 'accuracy'])

### Decision tree

In [168]:
dt_params = {'max_depth': [5, 10, 50, 100, 200],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10]}

In [169]:
dt_clf = GridSearchCV(tree.DecisionTreeClassifier(),
                      dt_params,
                      cv = skf,
                      n_jobs = -1,
                      scoring = ['f1', 'accuracy'],
                      refit = 'f1',
                      verbose = 1)
dt_clf.fit(X, y)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 50, 100, 200],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 10, 20]},
             refit='f1', scoring=['f1', 'accuracy'], verbose=1)

In [170]:
dt_f1_mean = dt_clf.cv_results_['mean_test_f1'][dt_clf.cv_results_['rank_test_f1'].argmin()]
dt_acc_mean = dt_clf.cv_results_['mean_test_accuracy'][dt_clf.cv_results_['rank_test_f1'].argmin()]

results.loc['Decision Tree'] = [dt_f1_mean, dt_acc_mean]

### Random forest

In [171]:
rf_params = {'n_estimators': [100, 200],
             'max_depth': [10, None],
             'min_samples_split': [2, 10],
             'min_samples_leaf': [1, 10]}

In [172]:
rf_clf = GridSearchCV(RandomForestClassifier(),
                      rf_params,
                      cv = skf,
                      n_jobs = -1,
                      scoring = ['f1', 'accuracy'],
                      refit = 'f1',
                      verbose = 1)
rf_clf.fit(X, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, None], 'min_samples_leaf': [1, 10],
                         'min_samples_split': [2, 10],
                         'n_estimators': [100, 200]},
             refit='f1', scoring=['f1', 'accuracy'], verbose=1)

In [173]:
rf_f1_mean = rf_clf.cv_results_['mean_test_f1'][rf_clf.cv_results_['rank_test_f1'].argmin()]
rf_acc_mean = rf_clf.cv_results_['mean_test_accuracy'][rf_clf.cv_results_['rank_test_f1'].argmin()]

results.loc['Random Forest'] = [rf_f1_mean, rf_acc_mean]

### xGboost

In [199]:
xgb_params = {'min_child_weight': [1, 5, 10],
              'gamma': [0.5, 1, 1.5, 2, 5],
              'subsample': [0.6, 0.8, 1.0],
              'colsample_bytree': [0.6, 0.8, 1.0],
              'max_depth': [3, 4, 5]}

In [200]:
xgb = XGBClassifier(learning_rate=0.02,
                    n_estimators=600,
                    objective='binary:logistic',
                    silent=True)

In [201]:
xgb_rs = RandomizedSearchCV(xgb,
                            param_distributions = xgb_params,
                            n_iter = 30,
                            scoring = ['f1', 'accuracy'],
                            refit = 'f1',
                            n_jobs = -1,
                            cv = skf,
                            verbose = 1)
xgb_rs.fit(X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\vladb\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\vladb\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\vladb\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\vladb\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version.

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importan...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=600, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 5],
                                        'max_depth': [3, 4, 5],
                                        'min_child_weight': [1, 5, 10],
                                        'subsample': [0.6, 0.8, 1.0]},
                   refit='f1', scoring=['f1', 'accuracy'], verbose=1)

In [202]:
xgb_f1_mean = xgb_rs.cv_results_['mean_test_f1'][xgb_rs.cv_results_['rank_test_f1'].argmin()]
xgb_acc_mean = xgb_rs.cv_results_['mean_test_accuracy'][xgb_rs.cv_results_['rank_test_f1'].argmin()]

results.loc['xGboost'] = [xgb_f1_mean, xgb_acc_mean]

### CatBoost

In [178]:
cat_features = X.select_dtypes(include=['object']).columns.tolist()
cat = CatBoostClassifier(cat_features=cat_features)

In [179]:
def cv_scores(cv_data):
    cv_data.head(10)

    best_acc_value = cv_data['test-Accuracy-mean'].max()
    best_acc_iter = cv_data['test-Accuracy-mean'].values.argmax()

    best_f1_value = cv_data['test-F1-mean'].max()
    best_f1_iter = cv_data['test-F1-mean'].values.argmax()
    return best_f1_value, best_acc_value

In [180]:
cat_params = {'loss_function': 'Logloss',
              'iterations': 100,
              'custom_loss': ['F1', 'Accuracy'],
              'learning_rate': 0.5,}

In [181]:
train_pool = Pool(data=X, label=y, cat_features=cat_features, has_header=True)
cv_data = cv(params = cat_params,
             pool = train_pool,
             verbose = False,
             folds = skf,
             plot=True)

cat_f1_mean, cat_acc_mean = cv_scores(cv_data)

results.loc['CatBoost'] = [cat_f1_mean, cat_acc_mean]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.2777165699
bestIteration = 18

Training on fold [1/5]

bestTest = 0.2253756756
bestIteration = 84

Training on fold [2/5]

bestTest = 0.2761589498
bestIteration = 50

Training on fold [3/5]

bestTest = 0.3054508481
bestIteration = 34

Training on fold [4/5]

bestTest = 0.3100233856
bestIteration = 14



### kNN

Since KNN cant handle categorical features, we will one-hot-encode every categorical feature and then reduce dimensionality with svd decomposition.

In [182]:
oh_enc = OneHotEncoder(drop = 'first')
X_oh = oh_enc.fit_transform(X)

In [183]:
decomp = TruncatedSVD()
knn = KNeighborsClassifier()

knn_pipe = Pipeline(steps=[('svd', decomp),
                           ('knn', knn)])

knn_params = {"svd__n_components": [2, 5, 10],
              "svd__n_iter": [5],
              "knn__n_neighbors": [1, 2, 3, 4, 5],
              "knn__weights" : ["uniform", "distance"],
              "knn__metric" : ["euclidean"]}

In [184]:
knn_clf = GridSearchCV(knn_pipe,
                      knn_params,
                      cv = skf,
                      n_jobs = -1,
                      scoring = ['f1', 'accuracy'],
                      refit = 'f1',
                      verbose = 1)

knn_clf.fit(X_oh, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('svd', TruncatedSVD()),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__metric': ['euclidean'],
                         'knn__n_neighbors': [1, 2, 3, 4, 5],
                         'knn__weights': ['uniform', 'distance'],
                         'svd__n_components': [2, 5, 10], 'svd__n_iter': [5]},
             refit='f1', scoring=['f1', 'accuracy'], verbose=1)

In [185]:
knn_f1_mean = knn_clf.cv_results_['mean_test_f1'][knn_clf.cv_results_['rank_test_f1'].argmin()]
knn_acc_mean = knn_clf.cv_results_['mean_test_accuracy'][knn_clf.cv_results_['rank_test_f1'].argmin()]

results.loc['KNN'] = [knn_f1_mean, knn_acc_mean]

### Naive Bayes

In [186]:
gnb = GaussianNB()
gnb_pipe = Pipeline(steps=[('svd', decomp),
                           ('gnb', gnb)])
gnb_params = {"svd__n_components": [2, 5, 10],
              "svd__n_iter": [5]}

In [187]:
gnb_clf = GridSearchCV(gnb_pipe,
                       gnb_params,
                       cv = skf,
                       n_jobs = -1,
                       scoring = ['f1', 'accuracy'],
                       refit = 'f1',
                       verbose = 1)

gnb_clf.fit(X_oh, y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('svd', TruncatedSVD()),
                                       ('gnb', GaussianNB())]),
             n_jobs=-1,
             param_grid={'svd__n_components': [2, 5, 10], 'svd__n_iter': [5]},
             refit='f1', scoring=['f1', 'accuracy'], verbose=1)

In [188]:
gnb_f1_mean = gnb_clf.cv_results_['mean_test_f1'][gnb_clf.cv_results_['rank_test_f1'].argmin()]
gnb_acc_mean = gnb_clf.cv_results_['mean_test_accuracy'][gnb_clf.cv_results_['rank_test_f1'].argmin()]

results.loc['GaussianNB'] = [gnb_f1_mean, gnb_acc_mean]

### Logistic Regression

In [189]:
lr = LogisticRegression(max_iter=1000)
lr_params = [{'penalty': ['l1', 'l2'],
              'C': [0.1, 1, 10],
              'solver': ['liblinear']},

             {'penalty': [None],
              'solver': ['lbfgs']},]

lr_clf = GridSearchCV(lr,
                      lr_params,
                      cv = skf,
                      n_jobs = -1,
                      scoring = ['f1', 'accuracy'],
                      refit = 'f1',
                      verbose = 1)

lr_clf.fit(X_oh, y)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid=[{'C': [0.1, 1, 10], 'penalty': ['l1', 'l2'],
                          'solver': ['liblinear']},
                         {'penalty': [None], 'solver': ['lbfgs']}],
             refit='f1', scoring=['f1', 'accuracy'], verbose=1)

In [190]:
lr_f1_mean = lr_clf.cv_results_['mean_test_f1'][lr_clf.cv_results_['rank_test_f1'].argmin()]
lr_acc_mean = lr_clf.cv_results_['mean_test_accuracy'][lr_clf.cv_results_['rank_test_f1'].argmin()]

results.loc['LogisticRegression'] = [lr_f1_mean, lr_acc_mean]

## Results

In [203]:
results

,f1,accuracy
Decision Tree,0.416982,0.859690
Random Forest,0.529041,0.925549
xGboost,0.320615,0.905791
CatBoost,0.522380,0.917633
KNN,0.412908,0.890648
GaussianNB,0.012121,0.893940
LogisticRegression,0.501999,0.903155


In [198]:
results.to_csv('bank_application_resuts.csv')

In [204]:
results.round(3)

,f1,accuracy
Decision Tree,0.417,0.860
Random Forest,0.529,0.926
xGboost,0.321,0.906
CatBoost,0.522,0.918
KNN,0.413,0.891
GaussianNB,0.012,0.894
LogisticRegression,0.502,0.903
